Importing modules:

In [547]:
import pandas as pd
import re
import Levenshtein as lev
import numpy as np
import nltk

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

import operator
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from IPython.display import Image

Read two datafiles: one is the individual-level placement (data2020-xx-xx); one is the university-level info (e.g. rank, location,etc).

In [538]:
filename = "data//data2020-05-30"
df = pd.read_csv(filename + '.csv')
plm_acr = pd.read_excel("placement_univ_acronym.xlsx")

## Cleaning the year column

I find that the format of year data is not the same (some integer, some float, some string, etc).

In [370]:
# Tabulate all year data
df['Year'].value_counts()

2018         365
2015         338
2017         325
2019         319
2016         312
2013         299
2014         284
2011         280
2012         259
2010         213
2008         172
2009         154
2020         152
2007         135
2006         110
2005         107
2004          48
2002          33
2003          30
2001          28
2000          26
1999          24
2016          17
2019-2020     16
1998           9
2011.0         9
1995           9
2006.0         8
1996           8
1994           7
2016.0         7
2019.0         7
1993           7
2018.0         7
2009.0         7
2017.0         7
2008.0         6
1997           6
1991           6
2010.0         5
2013.0         5
2015.0         5
2005.0         4
2007.0         4
2012.0         4
2020.0         4
2014.0         3
1992           2
2014           2
2017 Fall      1
Name: Year, dtype: int64

Manually correct the format and several outliers of the year data:

In [371]:
df = df[pd.isna(df['Year']) == False]
df[df['Year'] == '2019-2020'] = '2020'
df[df['Year'] == '2017\xa0Fall'] = '2017'
df['Year'] = df['Year'].astype(float).astype(int)

In [372]:
# Tabulate all year data
df['Year'].value_counts().sort_index(ascending = False)

2020    172
2019    326
2018    372
2017    333
2016    336
2015    343
2014    289
2013    304
2012    263
2011    289
2010    218
2009    161
2008    178
2007    139
2006    118
2005    111
2004     48
2003     30
2002     33
2001     28
2000     26
1999     24
1998      9
1997      6
1996      8
1995      9
1994      7
1993      7
1992      2
1991      6
Name: Year, dtype: int64

## Cleaning the placement data

The placement data requires more efforts to clean. Manually correct the below syntax:

In [374]:
df['Placement'] = df['Placement'].str.replace('–', ' ')
df['Placement'] = df['Placement'].str.replace('—', ' ')
df['Placement'] = df['Placement'].str.replace('^\s+', '', regex = True)
df['Placement'] = df['Placement'].str.replace('^<strong>', '', regex = True)
df['Placement'] = df['Placement'].str.replace('^<span>', '', regex = True)
df['Placement'] = df['Placement'].str.replace('^<a href=".*">', '', regex = True)
df['Placement'] = df['Placement'].str.replace('<.*>', '', regex = True)
df['Placement'] = df['Placement'].str.replace('&amp', ' ', regex = True)
df['Placement'] = df['Placement'].str.replace('<$', '', regex = True)
df['Placement'] = df['Placement'].str.replace('\($', '', regex = True)
df['Placement'] = df['Placement'].str.replace('\s+$', '', regex = True)
df['Placement'] = df['Placement'].str.replace('\xa0', ' ')
df = df[~df['Placement'].isnull()]

Use the nlp function to label entity:

In [375]:
# All unique placement
plm = df['Placement'].unique()

nlp = en_core_web_sm.load()
plm_map1 = {}

for i in plm:
    doc = nlp(i.lower())
    plm_map1[i] = [(X.text, X.label_) for X in doc.ents]

In [522]:
# only keep the entity labeled as 'ORG'
# some ORG examples include xxxx university, imf, world bank, etc
plm_org_li = [i[0][0] for i in list(plm_map1.values()) if i != [] and i[0][1] == 'ORG']
plm_org_li_freq = Counter(plm_org_li)
# del plm_org_li_freq['department of economics']
plm_org_li_count = Counter(plm_org_li_freq)
# print(plm_org_li_count.most_common())

Below implements the logics:

Step 1: if nlp returns no valid labeled entity, then use original placement name

Step 2: otherwise, check the number of labeled entities

Step 3: if the number of labeled entities have no ORG, then use original placement name

Step 4: if the number of labeled entities have just one ORG, use that ORG

Step 5: if the number of labeled entities have more than one ORG, use the one with highest frequency based on plm_org_li_freq

In [441]:
plm_cleaned1 = df.copy()
plm_cleaned1['cleanedPlacement'] = None

for i in plm_cleaned1.index:
    
    plm_name = plm_cleaned1['Placement'][i]
    doc = nlp(plm_name.lower())
    plm_doc = [(X.text, X.label_) for X in doc.ents]
    
    if len(plm_doc) == 0:
        plm_cleaned1.loc[i, 'cleanedPlacement'] = plm_name.lower()
        
    else:   
        
        org = [j[0] for j in plm_doc if j[1] == 'ORG']

        if len(org) > 0:
            
            num_org = [plm_org_li_freq[k] for k in org]
            idx = num_org.index(max(num_org))
            plm_cleaned1.loc[i, 'cleanedPlacement'] = org[idx]
        
        else:
            
            plm_cleaned1.loc[i, 'cleanedPlacement'] = plm_name.lower()

Identify the academic palcement:

Step 1: rule-based: all placement containing "university", "college", "school", etc

Step 2: from step 1, identify a set of school names, e.g. fudan univ -> fudan

Step 3: from plm_acr find the university name and acronym

Step 4: apply another rule, placement containing the school names from step 2 or university name or acronym from step 3

In [539]:
# remove the null cleaned placement
plm_cleaned1 = plm_cleaned1[~pd.isna(plm_cleaned1['cleanedPlacement'])]
plm_acr = plm_acr.rename(columns={'acronym': 'Acronym'}).drop(['link', 'note'], axis = 1)
univ, acr = list(plm_acr['university'].str.lower()), list(plm_acr['Acronym'])

In [544]:
plm_cleaned = pd.merge(plm_cleaned1, plm_acr, 'left', 'Acronym')

plm_cleaned['Academia'] = (plm_cleaned['cleanedPlacement'].str.contains('university')) | \
                            (plm_cleaned['cleanedPlacement'].str.contains('college')) | \
                            (plm_cleaned['cleanedPlacement'].str.contains('school')) | \
                            (plm_cleaned['cleanedPlacement'].str.contains('professor')) | \
                            (plm_cleaned['cleanedPlacement'].str.contains('postdoc'))

acr = list(plm_cleaned['cleanedPlacement'][plm_cleaned['Academia']].str.replace('university of ', '')\
        .str.replace(' university', '').str.replace(' ', ''))
acr.extend(list(plm_acr['Acronym']))

univ = list(plm_acr['university'].str.lower())

plm_cleaned['Academia1'] = plm_cleaned['cleanedPlacement'].isin(acr)
plm_cleaned['Academia2'] = plm_cleaned['cleanedPlacement'].isin(univ)
plm_cleaned['Academia'] = plm_cleaned['Academia'] | plm_cleaned['Academia1'] | plm_cleaned['Academia2']
plm_cleaned.drop(['Academia1', 'Academia2'], axis = 1, inplace = True)

plm_cleaned['Rank'] = plm_cleaned['usnews'].str[1:3]
# All NP are universities outside the 1-72 rank
plm_cleaned.loc[plm_cleaned['Rank'] == 'NP', 'Rank'] = '80'

plm_cleaned = plm_cleaned[~pd.isna(plm_cleaned['university'])]
plm_cleaned

,Year,Name,Placement,Acronym,cleanedPlacement,university,place,usnews,Academia,Rank
0,2019,<p>Yahia Abuhashem,Anthem,american,anthem,American University,"Washington, DC",RNP in Economics,False,80
1,2019,Zidong An,International Monetary Fund,american,international monetary fund,American University,"Washington, DC",RNP in Economics,False,80
2,2019,James Boohaker,US Government Accountability Office,american,us government accountability office,American University,"Washington, DC",RNP in Economics,False,80
3,2019,Woubet Kassa,World Bank,american,world bank,American University,"Washington, DC",RNP in Economics,False,80
4,2019,Natalia Leszczyszyn,Environmental Protection Agency,american,environmental protection agency,American University,"Washington, DC",RNP in Economics,False,80
...,...,...,...,...,...,...,...,...,...,...
4184,2009,Zhengzheng Pan,Facebook,vt,facebook,Virginia Tech,"Blacksburg, VA",#59 in Economics (tie),False,59
4185,2008,Jaideep Chowdhury,James Madison University,vt,james madison university,Virginia Tech,"Blacksburg, VA",#59 in Economics (tie),True,59
4186,2008,Andre Crawford,Pricewaterhouse Coopers,vt,coopers,Virginia Tech,"Blacksburg, VA",#59 in Economics (tie),False,59
4187,2007,Reza Kheirandish,Clayton State University School of Business,vt,clayton state university school of,Virginia Tech,"Blacksburg, VA",#59 in Economics (tie),True,59


In [546]:
plm_cleaned.to_csv(filename + "cleaned.csv", index = False)

## Other attempts to clean placement data (Not used)

Attempt 1: count the number of shared tokens in placement
For example: 'fudan university, SOE' and 'Assistant Prof, fudan', where 'fudan' is shared.

In [526]:
str1 = 'VP, Advanced Analytical Consultant at Wells Fargo'
d = {}

for i in plm:
    li1= str1.lower().split(' ')
    li2 = i.lower().split(' ')
    ins = list(set(li1).intersection(set(li2)))
    d[i] = len(ins)

In [527]:
sort_orders = sorted(d.items(), key=lambda x: x[1], reverse = True)

for i in sort_orders[0:10]:
    print(i[0], i[1])

VP, Advanced Analytical Consultant at Wells Fargo 7
Wells Fargo Marketing Database Consultant 3
Consultant at EconOne in Berkeley, California 2
Hanqing Advanced Institute of Economics and Finance at Renming University 2
United States Air Force (active duty, next assignment expected to at the Academy) 1
University of Nebraska at Kearney (tenure track) 1
Consultant to the World Bank Africa Gender Innovation Lab 1
Center for Education and the Workforce at Georgetown University 1
University at Buffalo, The State University of New York, (Assistant Professor) 1
School of Finance at Nankai University, China (assistant professor) 1


Attempt 2: Levenshtein distance method

In [528]:
str1 = 'fudan university'
d = {}

for i in plm:
    d[i] = lev.distance(str1, i.lower())

In [529]:
sort_orders = sorted(d.items(), key=lambda x: x[1])

for i in sort_orders[0:10]:
    print(i[0], i[1])

Fudan University 0
Furman University 2
Wuhan University 2
Jinan University 3
Tulane University 3
Fordham University 4
Brown University 4
Purdue University 4
Duke University 4
Auburn University 4
